# Análise Preditiva com Inteligência Artificial
## Detecção de Câncer de Pele com Reconhecimento de Imagens


O objetivo desta análise preditiva é identificar um câncer de pele através do reconhecimento de imagens e para isso será usado inteligência artificial, mais precisamente será utilizado uma CNN (Redes Neurais Convolucionais) para a deteccção desta doença.

A fonte dos dados é: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

## 1. Introdução

A pele, o maior órgão do corpo humano, é composta por duas camadas: a epiderme, na parte externa, e a derme, na parte interna. Além de regular a temperatura do corpo, ela serve de proteção contra agentes externos, como luz do sol e calor, contra agentes infecciosos e agentes químicos (ingestão de arsênico, exposição a raios-X e rádio).

O câncer de pele é muito comuns no Brasil (25% dos tumores malignos diagnosticados), e ocorre por causa do excesso de exposição aos raios ultravioleta do sol.

Eles podem ser de vários tipos de câncer de pele. Os mais comuns são os carcinomas (carcinoma basocelular e carcinoma epidermoide) com incidência mais alta, porém menor gravidade, e os melanomas que, apesar de menos frequentes, são mais graves por causa do risco de metástases aumentado.

Fonte: https://drauziovarella.uol.com.br/doencas-e-sintomas/cancer-de-pele/

## 2. Instalação e Carga de Pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem Python usada neste Jupyter Notebook é: ', python_version())

A versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [2]:
# Instala o Pytorch
#!pip install -q torch==1.13.0

In [3]:
# Instala o Lightning
#!pip install -q pytorch0lightning==1.8.3

In [6]:
# Pacotes

# Manipulação de dados e imagens
import os
import cv2
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Pytorch
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

# Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources
import pytorch_lightning as pl

# Seeds
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Lab04 - Visão Computacional" --iversions

Author: Lab04 - Visão Computacional

numpy            : 1.21.5
torchvision      : 0.14.0
PIL              : 9.2.0
torch            : 1.13.0
pytorch_lightning: 1.8.3
cv2              : 4.7.0
pandas           : 1.4.4
matplotlib       : 3.5.2



## 3. Verificando o Ambiente de Desenolvimento

In [8]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print("\nModelo da GPU:")
# Modelo da GPU usada
!nvidia-smi --query-gpu=name --format=csv,noheader
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('Pillow', '9.2.0'), ('tqdm', '4.64.1'), ('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('pandas', '1.4.4'), ('torchvision', '0.14.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

Modelo da GPU:


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.



------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## 4. Mapeamento das Imagens

In [9]:
# Pasta com as imagens
pasta_imagens = 'input'

In [10]:
# Obtém o caminho de cada imagem
caminho_imagens = glob(os.path.join(pasta_imagens, '*', '*.jpg'))

In [11]:
# Cria um dicionário com o ID da imagem e o caminho na pasta
dict_map_imagem_caminho = {os.path.splitext(os.path.basename(x))[0]: x for x in caminho_imagens}

# Visualiza o dicionário
dict_map_imagem_caminho

{'ISIC_0026306': 'input\\HAM10000_images_part_1\\ISIC_0026306.jpg',
 'ISIC_0026307': 'input\\HAM10000_images_part_1\\ISIC_0026307.jpg',
 'ISIC_0026308': 'input\\HAM10000_images_part_1\\ISIC_0026308.jpg',
 'ISIC_0026309': 'input\\HAM10000_images_part_1\\ISIC_0026309.jpg',
 'ISIC_0026310': 'input\\HAM10000_images_part_1\\ISIC_0026310.jpg',
 'ISIC_0026311': 'input\\HAM10000_images_part_1\\ISIC_0026311.jpg',
 'ISIC_0026312': 'input\\HAM10000_images_part_1\\ISIC_0026312.jpg',
 'ISIC_0026313': 'input\\HAM10000_images_part_1\\ISIC_0026313.jpg',
 'ISIC_0026314': 'input\\HAM10000_images_part_1\\ISIC_0026314.jpg',
 'ISIC_0026315': 'input\\HAM10000_images_part_1\\ISIC_0026315.jpg',
 'ISIC_0026316': 'input\\HAM10000_images_part_1\\ISIC_0026316.jpg',
 'ISIC_0026317': 'input\\HAM10000_images_part_1\\ISIC_0026317.jpg',
 'ISIC_0026318': 'input\\HAM10000_images_part_1\\ISIC_0026318.jpg',
 'ISIC_0026319': 'input\\HAM10000_images_part_1\\ISIC_0026319.jpg',
 'ISIC_0026320': 'input\\HAM10000_images_part_1\

In [12]:
# Leitura do arquivo de metadados
df_original = pd.read_csv(os.path.join(pasta_imagens, 'HAM10000_metadata.csv'))

# Visualiza a base
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [13]:
# Adiciona o path na base
df_original['path'] = df_original['image_id'].map(dict_map_imagem_caminho.get)

# Visualiza a base
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,None
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,None
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,input\HAM10000_images_part_2\ISIC_0031633.jpg


Como a base é muito grande e o processamento será em CPU, alguns registros apareceram como none. 

In [14]:
# Tipos de lesões que serão analisados
dict_tipo_lesao = {'nv': 'Melanocytic nevi',
                   'mel': 'dermatofibroma',
                   'bkl': 'Benign keratosis-like lesions',
                   'bcc': 'Basal cell carcinoma',
                   'akiec': 'Actinic keratoses',
                   'vasc': 'Vascular lesions',
                   'df': 'Dermatofibroma'}

In [15]:
# Inclui o tipo de lesão na base
df_original['cell_type'] = df_original['dx'].map(dict_tipo_lesao.get)

# Visualiza os dados
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like lesions
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like lesions
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,input\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like lesions


In [16]:
# Inclui o index do dicionário na base
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes

# Visualiza os dados
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,input\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,input\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like lesions,2
